In [1]:
import pandas as pd
import chromadb

#### Load
Preprocessed Data

In [2]:
df = pd.read_csv("../data/2024/cleaned_data.csv", sep=';')


Player Information

In [3]:
data = pd.read_csv('../data/2024/male_players.csv')
data = data[data['fifa_version'] == 24.0]
player_information = ['player_id', 'short_name', 'age', 'overall', 'player_positions','preferred_foot' , 'nationality_name','club_name','value_eur','wage_eur', ]
df_player_information = data[player_information]
display(df_player_information)

C:\Users\Alexander Lorenz\AppData\Local\Temp\ipykernel_25776\1479636954.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/2024/male_players.csv')


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
0,231747,K. Mbappé,24,91,"ST, LW",Right,France,Paris Saint Germain,181500000.0,230000.0
1,239085,E. Haaland,22,91,ST,Left,Norway,Manchester City,185000000.0,340000.0
2,192985,K. De Bruyne,32,91,"CM, CAM",Right,Belgium,Manchester City,103000000.0,350000.0
3,158023,L. Messi,36,90,"CF, CAM",Left,Argentina,Inter Miami,41000000.0,23000.0
4,165153,K. Benzema,35,90,"CF, ST",Right,France,Al Ittihad,51000000.0,95000.0
...,...,...,...,...,...,...,...,...,...,...
18345,273759,S. Telem,20,47,CB,Left,India,Jamshedpur,100000.0,500.0
18346,269013,Jin Liangkuan,21,47,CB,Right,China PR,Meizhou Hakka,100000.0,1000.0
18347,272748,M. Mewlan,19,47,ST,Right,China PR,Shandong Taishan,100000.0,2000.0
18348,269534,Yin Jie,21,47,CM,Right,China PR,Zhejiang,90000.0,1000.0


#### Prepare Data and get embeddings

In [4]:
ids = df['player_id'].astype(str).tolist()
df_cleaned = df.drop(columns=['player_id', 'short_name'])
embeddings = df_cleaned.values.tolist()

#### Vector Database

In [5]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(
    name = "player_database",
    metadata={"hnsw:space": "cosine"}
)

#### Add items

In [6]:
collection.add(
    embeddings= embeddings,
    #metadatas=[{"source": "source a"}, {"source": "source b"}],
    ids=ids
)

#### Query

In [10]:
### custom target
target = df[df['player_id'] == 244176]

# prepare target
target_id = target['player_id'].iloc[0]
target_cleaned = target.drop(columns=['player_id', 'short_name'])
target_embedding = target_cleaned.values.tolist()

In [11]:
query_player = target_embedding

results = collection.query(
    query_embeddings=query_player,
    n_results=10
)
#

# display query object
print("Queried Player")
display(df_player_information[df_player_information['player_id'] == target_id])

query_idx = list(results.get('ids')[0])

# output player names
print("Similar players")


for idx in query_idx:
    id_int = int(idx)
    display(df_player_information[df_player_information['player_id'] == id_int])


Queried Player


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
1595,244176,D. Undav,26,75,"ST, CAM",Right,Germany,VfB Stuttgart,6500000.0,69000.0


Similar players


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
1595,244176,D. Undav,26,75,"ST, CAM",Right,Germany,VfB Stuttgart,6500000.0,69000.0


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
901,232432,L. Jović,25,77,ST,Right,Serbia,Milan,15000000.0,58000.0


...

,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
1813,262642,Z. Amdouni,22,74,"ST, CAM",Right,Switzerland,Burnley,9500000.0,37000.0


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
1780,251412,S. Weissman,27,75,ST,Right,Israel,Granada,6000000.0,21000.0


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
1567,215247,M. Ingvartsen,27,75,"CF, ST, CAM",Left,Denmark,Nordsjælland,6500000.0,16000.0


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
456,205431,N. Füllkrug,30,80,ST,Right,Germany,Borussia Dortmund,20500000.0,56000.0


,player_id,short_name,age,overall,player_positions,preferred_foot,nationality_name,club_name,value_eur,wage_eur
2166,215061,D. Benedetto,33,74,ST,Right,Argentina,Boca Juniors,2700000.0,18000.0


In [9]:
distances = results['distances'][0] 
for dist in distances:
    similarity_percentage = ((1 - dist) * 100)
    print(f"Similarity percentage with base query item: {similarity_percentage:.2f}%")


Similarity percentage with base query item: 100.00%
Similarity percentage with base query item: 98.98%
Similarity percentage with base query item: 98.85%
Similarity percentage with base query item: 98.85%
Similarity percentage with base query item: 98.84%
Similarity percentage with base query item: 98.81%
Similarity percentage with base query item: 98.81%
Similarity percentage with base query item: 98.79%
Similarity percentage with base query item: 98.77%
Similarity percentage with base query item: 98.77%


#### TODO
- Similarity Comparison to FC Bayern IV

#### Learnings
- Position kann manchmal Ärger machen
- Alter sollte ein Filter sein
- Empfehlungen sind nicht immer absehbar